In [1]:
import pandas as pd
import os
import re
from tqdm import tqdm
dataset_name = 'en'
input_dir = f"annotated_gpt/inputs/{dataset_name}"
output_dir = f"annotated_gpt/outputs/{dataset_name}"

all_data_task_topk = pd.read_csv(f'annotated_gpt/inputs/{dataset_name}.csv', sep='\t').reset_index()


final_ans = {'index':[], 'gpt_task': []}
for i in tqdm(os.listdir(output_dir)):
    idx = int(re.findall(r'\d+', i)[0])
    text = open(f"{output_dir}/{idx}.txt", 'r').read()
    content = text
    text = open(f"{output_dir}/{idx}.txt", 'r').read()
    ans = re.findall(r'\d\.+(.*?)\n', content)
    candidates = [all_data_task_topk[f'task{_}'][idx].strip(' ') for _ in range(100)]
    results = []
    # rgex = '|'.join([_ for _ in  candidates])
    # results = re.findall(rf"{rgex}", content)
    for line in content.split('\n'):
        for can in candidates:
            if can in line:
                results.append(can)
                break
    if len(results) != 10:
        continue
    final_ans['gpt_task'].append(results)
    final_ans['index'].append(idx)
final_ans = pd.DataFrame(final_ans)
# 处理输出文件
final_ans = pd.merge(all_data_task_topk[['index', 'job_title', 'job_description', 'task']], final_ans, on=['index'])
final_ans.to_csv(f'annotated_gpt/outputs/{dataset_name}.csv', index=None)
final_ans

100%|██████████| 195/195 [00:01<00:00, 111.41it/s]


,index,job_title,job_description,task,gpt_task
0,0,Digital Marketing Manager,Develop and implement digital marketing strate...,Develop marketing plans or strategies.,[Develop content for sales presentations or ot...
1,1,Senior Manager Category Development,"Deliver relevant, localized category insights ...",Develop marketing plans or strategies.,"[Analyze consumer trends., Analyze market rese..."
2,2,Sales Account Executive,Identify client’s pain points and create probl...,Develop marketing plans or strategies.,"[Develop promotional strategies or plans., Ide..."
3,3,Regional Marketing Manager,"Create and execute targeted, thematic marketin...",Develop marketing plans or strategies.,"[Develop marketing plans or strategies., Devel..."
4,4,Marketing Director,Communicate weekly with other Directors (Sales...,Develop marketing plans or strategies.,"[Develop marketing plans or strategies., Liais..."
...,...,...,...,...,...
166,185,Chief of Staff to the Chief Legal Officer,"Ensure the team is a smoothly running engine, ...",Liaise between departments or other groups to ...,[Liaise between departments or other groups to...
167,188,Production Assistant - Wholesale Insurance Bro...,Work closely with AB Team Lead and assigned Br...,Liaise between departments or other groups to ...,[Liaise between departments or other groups to...
168,190,"Head of Direct Procurement, Global",Managing a large international team across 18+...,Liaise between departments or other groups to ...,"[Manage human resources activities., Liaise be..."
169,192,"Director of Finance - The Langham, New York",Direct accounting staff to ensure accurate acc...,Develop financial or business plans.,"[Direct sales, marketing, or customer service ..."


In [4]:
# final_ans = final_ans[final_ans['index']>=147]

final_ans['gpt_task'] = final_ans['gpt_task'].apply(lambda x: x[0])
final_group = final_ans.groupby('gpt_task').agg(list)[['job_description']]
final_group['len'] = final_group['job_description'].apply(len)
final_group = final_group.sort_values('len', ascending=False)
final_group = final_group.explode('job_description').reset_index()[['gpt_task', 'job_description', 'len']]
final_group = pd.merge(final_group, final_ans, on=['gpt_task', 'job_description'])[['job_title', 'job_description', 'gpt_task', 'task', 'len']]
if not os.path.exists(f'annotated_gpt/results'):
    os.makedirs(f'annotated_gpt/results')
final_group.to_excel(f'annotated_gpt/results/{dataset_name}.xlsx', index=None)
final_group

,job_title,job_description,gpt_task,task,len
0,"VP, Sr Retail Bus Banker - Illinois Market","Spend majority of time in the field, on sales ...",Contact current or potential customers to prom...,Contact current or potential customers to prom...,18
1,Associate Account Executive,Assist in executing marketing client campaigns...,Contact current or potential customers to prom...,Contact current or potential customers to prom...,18
2,Aftermarket Sales Representative,Develop a territory management plan to maximiz...,Contact current or potential customers to prom...,Contact current or potential customers to prom...,18
3,Commercial Lines Producer,"Prospect new business primarily by phone, mail...",Contact current or potential customers to prom...,Contact current or potential customers to prom...,18
4,Regional Sales Development Manager,Effectively communicate voice of customer (VOC...,Contact current or potential customers to prom...,Contact current or potential customers to prom...,18
...,...,...,...,...,...
166,Customer Experience Sales Guide,Working out financing options for the customer...,Discuss account status or activity with custom...,Contact current or potential customers to prom...,1
167,Customer Success Manager,Establish a trusted and strategic advisory rel...,Establish organizational guidelines or policies.,Communicate with customers to resolve complain...,1
168,Senior Manager/Director of Accounting Projects...,"• Take charge of accounting policies, includin...",Examine financial records to ensure compliance...,"Prepare financial documents, reports, or budgets.",1
169,Brand Marketing Manager (Restaurant),"Partner with the VP, Marketing and Director, B...",Gather organizational performance information.,Develop business or market strategies.,1


In [5]:
# import pandas as pd
# df1 = pd.read_excel('annotated_gpt/results/en.xlsx')
# df2 = pd.read_excel('annotated_gpt/results/en-annotated.xlsx')
# pd.merge(df1, df2, how='left').to_excel('annotated_gpt/results/en-annotated1.xlsx', index=None)